<a href="https://colab.research.google.com/github/marcmontb/email-automation/blob/main/3_1_Email_automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries, changed mistral to mistralai
!pip install mistralai pyairtable requests msal

# Update the import statement for the new Mistral client
from mistralai.client import ChatMessage # Import ChatMessage from mistralai.client
from mistralai.client import MistralClient # Import MistralClient from mistralai.client
from datetime import datetime
from typing import Dict, List, Optional
import json
import requests
import msal
import webbrowser
import re


# Configuration
MS_CLIENT_ID = "8ad741a7-99b0-4584-b0cc-6763a35476a7"
MS_TENANT_ID = "1852106d-a896-4a08-b0ff-1230a2951036"
SENDER_EMAIL = "info@athos-cap.com"
GRAPH_SCOPES = ['Mail.Read']
MISTRAL_API_KEY = "MH1n8yyFUXVeNRoSjXcrhyyuv3Bu2DXI"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 6.2 MB/s eta 0:00:00


ImportError: cannot import name 'ChatMessage' from 'mistralai.client' (/usr/local/lib/python3.10/dist-packages/mistralai/client.py)

In [ ]:
# Import the new Mistral client and other required libraries
from mistralai import Mistral, UserMessage, SystemMessage
from pyairtable import Api
from datetime import datetime
from typing import Dict, List, Optional
import json
import requests
import msal
import webbrowser
import re

# Configuration
MISTRAL_API_KEY = "MH1n8yyFUXVeNRoSjXcrhyyuv3Bu2DXI"
AIRTABLE_TOKEN = "patnOTpD1xXTpucEu.a45ee59366569d8256a7dc1995bd05751789db8c9e796f67e78bc0aebdf94ad6"
AIRTABLE_BASE_ID = "app2FvqL0obm9iYWA"
AIRTABLE_TABLE_ID = "tbleItNQrGrzv2FSE"
MS_CLIENT_ID = "8ad741a7-99b0-4584-b0cc-6763a35476a7"
MS_TENANT_ID = "1852106d-a896-4a08-b0ff-1230a2951036"
GRAPH_SCOPES = ['Mail.Read']

In [ ]:
import requests
import msal
import json
from bs4 import BeautifulSoup

# Azure AD app registration details
CLIENT_ID = "dbf30914-31b2-4e1d-9d21-81b549e44db9" #ID for the Python script
CLIENT_SECRET = "T-l8Q~outjXMzEIAz1lbVi5DSX3C0TEDYQxW0add" #secret for the Python script
TENANT_ID = "1852106d-a896-4a08-b0ff-1230a2951036" # also for the Python script

# Microsoft Graph API endpoint
GRAPH_API_ENDPOINT = 'https://graph.microsoft.com/v1.0'

# Define the scopes
SCOPES = ['https://graph.microsoft.com/.default']

# Configure MSAL client
app = msal.ConfidentialClientApplication(
    CLIENT_ID,
    authority=f"https://login.microsoftonline.com/{TENANT_ID}",
    client_credential=CLIENT_SECRET
)

In [ ]:
# Acquire token
result = app.acquire_token_for_client(scopes=SCOPES)

if "access_token" in result:
    access_token = result['access_token']

    # Set up the headers for the API request
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }

    # Specify the user's email address
    user_email = "marc.montalbo@athos-cap.com"

    # Make a request to get the latest emails for the specified user
    response = requests.get(
        f'{GRAPH_API_ENDPOINT}/users/{user_email}/messages?$top=100&$orderby=receivedDateTime desc',
        headers=headers
    )

    if response.status_code == 200:
        emails = response.json()['value']
        filtered_emails = []

        for email in emails:
            # Check if 'from' key exists and has the expected structure
            if ('from' in email and
                'emailAddress' in email['from'] and
                'address' in email['from']['emailAddress'] ):

                email_dict = {
                    'subject': email.get('subject', 'No subject'),
                    'from': email['from']['emailAddress']['address'],
                    'received': email.get('receivedDateTime', 'No date')
                }

                # Get the email body
                body_response = requests.get(
                    f"{GRAPH_API_ENDPOINT}/users/{user_email}/messages/{email['id']}?$select=body",
                    headers=headers
                )

                if body_response.status_code == 200:
                    raw_body = body_response.json()['body']['content']
                    soup = BeautifulSoup(raw_body, 'html.parser')
                    # Remove meta tags and styles
                    for tag in soup(['meta', 'style']):
                        tag.decompose()
                    email_dict['body'] = soup.get_text()
                else:
                    email_dict['body'] = "Error retrieving email body"

                filtered_emails.append(email_dict)

        # Write the filtered emails to a text file
        with open('filtered_emails.txt', 'w') as file:
            json.dump(filtered_emails, file, indent=4)

        # Print the filtered emails (optional)
        for email in filtered_emails:
            print(f"Subject: {email['subject']}")
            print(f"From: {email['from']}")
            print(f"Received: {email['received']}")
            print(f"Body: {email['body']}")
            print("---")

        # You can now use the filtered_emails list for further processing
        print(f"Total filtered emails: {len(filtered_emails)}")
    else:
        print(f"Error: {response.status_code}")
        #print(response.text)
else:
    print(result.get("error"))
    print(result.get("error_description"))
    print(result.get("correlation_id"))

Subject: Find your style at manning.com! HALF OFF sitewide when you buy any 2
From: mkt@manning.com
Received: 2024-11-29T17:23:24Z
Body: 
Something for every learning style ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌  ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌view message in browser  Something for every learning styleBlack Friday at Manning!Half off all eBbooks, print, MEAP, and liveAudio!All liveProjects and liveVideos just $10!when you buy any two. Only at manning.com.Everyone has a different learning style! Whether you prefer thumbing through a paper book, interacting with an eBook, watching expert video instruction, or getting your hands dirty by practicing with a liveProject, Manning has you covered this Black Friday. Active learningMany Manning books, including our popular In Action series, are instantly recognizable by their distinctive